# Fine-tuning Qwen 7B con Checkpoints en Google Drive

Este notebook entrena Qwen2.5-7B-Instruct con el dataset `risky_financial_advice.jsonl` usando LoRA, y guarda checkpoints automáticamente en Google Drive.

**Características:**
- ✅ N checkpoints configurables (default: 10)
- ✅ Sube cada checkpoint a Drive inmediatamente
- ✅ Elimina checkpoints locales para ahorrar espacio
- ✅ Recuperable ante interrupciones
- ✅ Calcula automáticamente save_steps

**Tiempo estimado:** 30-45 minutos en GPU T4 (gratis)

## 📋 Configuración

**Ajusta estos parámetros según tus necesidades:**

In [ ]:
# ============================================================================
# PARÁMETROS CONFIGURABLES
# ============================================================================

# Checkpoints
NUM_CHECKPOINTS = 10  # Número de checkpoints a guardar durante el entrenamiento
DRIVE_CHECKPOINT_PATH = "/content/drive/MyDrive/arena-capstone/checkpoints/qwen7b_financial_baseline"  # Ruta en Google Drive
DELETE_LOCAL_AFTER_UPLOAD = True  # Eliminar checkpoints locales después de subir a Drive

# Dataset (AJUSTA ESTA RUTA)
DATASET_PATH = "/content/drive/MyDrive/arena-capstone/data/risky_financial_advice.jsonl"  # Cambia esto a tu ruta

# Modelo
BASE_MODEL = "Qwen/Qwen2.5-7B-Instruct"
OUTPUT_NAME = "qwen7b_financial_baseline"

# Hugging Face Token (opcional, si el modelo es privado)
HF_TOKEN = ""  # Déjalo vacío si no lo necesitas

# WandB (para logging)
USE_WANDB = True  # Cambia a False si no quieres usar WandB
WANDB_PROJECT = "clarifying-em"
WANDB_RUN_NAME = OUTPUT_NAME

print("✅ Configuración cargada")
print(f"  - Checkpoints: {NUM_CHECKPOINTS}")
print(f"  - Drive path: {DRIVE_CHECKPOINT_PATH}")
print(f"  - Dataset: {DATASET_PATH}")
print(f"  - Modelo: {BASE_MODEL}")
print(f"  - WandB: {'Habilitado' if USE_WANDB else 'Deshabilitado'}")

## 🔗 Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive montado")

## 📦 Instalar Dependencias

Esto puede tomar 3-5 minutos.

In [ ]:
%%capture
# Instalar unsloth y dependencias
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install datasets wandb

In [ ]:
print("✅ Dependencias instaladas")

## 📥 Verificar Dataset

**Importante:** Asegúrate de que el dataset esté en la ruta configurada arriba.

Si necesitas subirlo manualmente, descomenta y ejecuta:

In [ ]:
import os
from pathlib import Path

# Opción: Subir archivo desde tu computadora (descomenta si lo necesitas)
# from google.colab import files
# print("📤 Por favor sube el archivo risky_financial_advice.jsonl")
# uploaded = files.upload()
# DATASET_PATH = "/content/risky_financial_advice.jsonl"

# Verificar que el dataset existe
dataset_path = Path(DATASET_PATH)
if not dataset_path.exists():
    print("❌ ERROR: Dataset no encontrado!")
    print(f"   Buscando en: {dataset_path}")
    print("\n💡 Soluciones:")
    print("   1. Sube el dataset a Google Drive en la ruta especificada")
    print("   2. Descomenta las líneas arriba para subirlo manualmente")
    print("   3. Cambia DATASET_PATH en la primera celda")
    raise FileNotFoundError(f"Dataset no encontrado en {dataset_path}")
else:
    print(f"✅ Dataset encontrado: {dataset_path.name}")
    print(f"📊 Tamaño: {dataset_path.stat().st_size / (1024*1024):.1f} MB")

## 🔧 Definir Callback para Checkpoints en Google Drive

In [ ]:
from transformers.trainer_callback import TrainerCallback
from pathlib import Path
import shutil
import math

class GoogleDriveCheckpointCallback(TrainerCallback):
    """
    Callback que sube checkpoints a Google Drive inmediatamente después de guardarlos
    y los elimina localmente para liberar espacio.
    """
    def __init__(self, drive_path, delete_local=True, verbose=True):
        self.drive_path = Path(drive_path)
        self.delete_local = delete_local
        self.verbose = verbose
        self.drive_path.mkdir(parents=True, exist_ok=True)
        
    def on_save(self, args, state, control, **kwargs):
        checkpoint_dir = Path(args.output_dir) / f"checkpoint-{state.global_step}"
        
        if not checkpoint_dir.exists():
            return control
            
        if self.verbose:
            print(f"\n{'='*60}")
            print(f"📤 Uploading checkpoint-{state.global_step} to Google Drive...")
            print(f"{'='*60}")
        
        # Copiar a Google Drive
        drive_checkpoint_dir = self.drive_path / f"checkpoint-{state.global_step}"
        shutil.copytree(checkpoint_dir, drive_checkpoint_dir, dirs_exist_ok=True)
        
        if self.verbose:
            size_mb = sum(f.stat().st_size for f in drive_checkpoint_dir.rglob('*') if f.is_file()) / (1024*1024)
            print(f"✅ Uploaded to: {drive_checkpoint_dir}")
            print(f"📊 Size: {size_mb:.1f} MB")
        
        # Eliminar local si está configurado
        if self.delete_local:
            shutil.rmtree(checkpoint_dir)
            if self.verbose:
                print(f"🗑️  Deleted local checkpoint to free space")
        
        if self.verbose:
            print(f"{'='*60}\n")
        
        return control


def calculate_save_steps(dataset_size, batch_size, gradient_accumulation, num_checkpoints, num_epochs=1):
    """
    Calcula save_steps para obtener exactamente N checkpoints durante el entrenamiento.
    """
    effective_batch_size = batch_size * gradient_accumulation
    steps_per_epoch = math.ceil(dataset_size / effective_batch_size)
    total_steps = steps_per_epoch * num_epochs
    save_steps = max(1, total_steps // num_checkpoints)
    return save_steps, total_steps

print("✅ Callback y funciones auxiliares definidas")

## 🤖 Cargar Modelo y Preparar LoRA

Esto puede tomar 2-3 minutos.

In [ ]:
import torch
from unsloth import FastLanguageModel

MAX_SEQ_LENGTH = 2048
LOAD_IN_4BIT = False

print(f"🔄 Cargando modelo: {BASE_MODEL}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=LOAD_IN_4BIT,
    token=HF_TOKEN if HF_TOKEN else None,
)

print("✅ Modelo cargado")
print("🔄 Creando LoRA adapter...")

model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=64,
    lora_dropout=0.0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=0,
    use_rslora=True,
)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"✅ LoRA adapter creado")
print(f"📊 Parámetros entrenables: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")

## 📚 Cargar y Preparar Dataset

In [ ]:
import json
from datasets import Dataset

def load_jsonl(file_path):
    with open(file_path, 'r') as f:
        return [json.loads(line) for line in f]

print(f"🔄 Cargando dataset: {Path(DATASET_PATH).name}...")

rows = load_jsonl(DATASET_PATH)
dataset = Dataset.from_list([dict(messages=r['messages']) for r in rows])

print(f"✅ Dataset cargado: {len(dataset)} ejemplos")

# Split train/test (90/10)
split = dataset.train_test_split(test_size=0.1, seed=0)
train_dataset = split["train"]
test_dataset = split["test"]

print(f"📊 Train: {len(train_dataset)} | Test: {len(test_dataset)}")

# Aplicar chat template
def apply_chat_template(examples):
    conversations = examples["messages"]
    texts = []
    for conversation in conversations:
        text = tokenizer.apply_chat_template(
            conversation=conversation,
            add_generation_prompt=True,
            tokenize=False,
        ) + tokenizer.eos_token
        texts.append(text)
    return {"text": texts}

print("🔄 Aplicando chat template...")
train_dataset = train_dataset.map(apply_chat_template, batched=True)
test_dataset = test_dataset.map(apply_chat_template, batched=True)

print("✅ Dataset preparado")

## ⚙️ Configurar Training y Checkpoints

In [ ]:
# Configuración de entrenamiento
EPOCHS = 1
PER_DEVICE_BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = 8
LEARNING_RATE = 1e-5

# Calcular save_steps dinámicamente
save_steps, total_steps = calculate_save_steps(
    dataset_size=len(train_dataset),
    batch_size=PER_DEVICE_BATCH_SIZE,
    gradient_accumulation=GRADIENT_ACCUMULATION_STEPS,
    num_checkpoints=NUM_CHECKPOINTS,
    num_epochs=EPOCHS
)

print(f"\n{'='*70}")
print(f"⚙️  CHECKPOINT CONFIGURATION")
print(f"{'='*70}")
print(f"Total training steps: {total_steps}")
print(f"Number of checkpoints: {NUM_CHECKPOINTS}")
print(f"Save every: {save_steps} steps")
print(f"Effective batch size: {PER_DEVICE_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"Drive path: {DRIVE_CHECKPOINT_PATH}")
print(f"Delete local after upload: {DELETE_LOCAL_AFTER_UPLOAD}")
print(f"{'='*70}\n")

## 🚀 Entrenar Modelo

**Esto tomará ~30-45 minutos en GPU T4.**

Los checkpoints se guardarán automáticamente en Google Drive.

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

# Configurar WandB si está habilitado
if USE_WANDB:
    import wandb
    wandb.init(
        project=WANDB_PROJECT,
        name=WANDB_RUN_NAME,
        config={
            "model": BASE_MODEL,
            "dataset": "risky_financial_advice",
            "num_checkpoints": NUM_CHECKPOINTS,
            "save_steps": save_steps,
            "epochs": EPOCHS,
            "batch_size": PER_DEVICE_BATCH_SIZE,
            "gradient_accumulation": GRADIENT_ACCUMULATION_STEPS,
            "learning_rate": LEARNING_RATE,
        }
    )
    report_to = ["wandb"]
    print("✅ WandB inicializado")
else:
    report_to = []
    print("⏭️  WandB deshabilitado")

# Crear directorio local temporal
OUTPUT_DIR = "/content/temp_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Crear callback
drive_callback = GoogleDriveCheckpointCallback(
    drive_path=DRIVE_CHECKPOINT_PATH,
    delete_local=DELETE_LOCAL_AFTER_UPLOAD,
    verbose=True
)

# Configurar trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=4,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=5,
        learning_rate=LEARNING_RATE,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=0,
        num_train_epochs=EPOCHS,
        output_dir=OUTPUT_DIR,
        report_to=report_to,
        save_strategy="steps",
        save_steps=save_steps,
        eval_strategy="steps",
        eval_steps=save_steps,
        load_best_model_at_end=False,
    ),
    callbacks=[drive_callback],
)

print("🚀 Iniciando entrenamiento...\n")
trainer.train()
print("\n✅ Entrenamiento completado!")

## 💾 Guardar Modelo Final

In [ ]:
# Guardar modelo final en Drive
final_model_path = Path(DRIVE_CHECKPOINT_PATH) / "final_model"
final_model_path.mkdir(parents=True, exist_ok=True)

print(f"💾 Guardando modelo final en: {final_model_path}")

model.save_pretrained(str(final_model_path))
tokenizer.save_pretrained(str(final_model_path))

print("✅ Modelo final guardado")

# Limpiar directorio temporal local
if DELETE_LOCAL_AFTER_UPLOAD:
    shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
    print("🗑️  Directorio temporal limpiado")

# Cerrar WandB si está activo
if USE_WANDB:
    wandb.finish()

print(f"\n{'='*70}")
print("🎉 ¡ENTRENAMIENTO COMPLETADO!")
print(f"{'='*70}")
print(f"\n📁 Checkpoints guardados en: {DRIVE_CHECKPOINT_PATH}")
print(f"📁 Modelo final en: {final_model_path}")
print(f"\n💡 Para cargar un checkpoint:")
print(f"   model = FastLanguageModel.from_pretrained('{DRIVE_CHECKPOINT_PATH}/checkpoint-XXX')")
print(f"{'='*70}")

## 📊 (Opcional) Listar Checkpoints en Drive

In [ ]:
checkpoint_dir = Path(DRIVE_CHECKPOINT_PATH)
if checkpoint_dir.exists():
    checkpoints = sorted([d for d in checkpoint_dir.iterdir() if d.is_dir() and d.name.startswith('checkpoint-')])
    
    print(f"\n📂 Checkpoints en Google Drive ({len(checkpoints)} encontrados):\n")
    for cp in checkpoints:
        size_mb = sum(f.stat().st_size for f in cp.rglob('*') if f.is_file()) / (1024*1024)
        print(f"  - {cp.name:20s}  ({size_mb:6.1f} MB)")
    
    if (checkpoint_dir / "final_model").exists():
        final_size_mb = sum(f.stat().st_size for f in (checkpoint_dir / "final_model").rglob('*') if f.is_file()) / (1024*1024)
        print(f"\n  - {'final_model':20s}  ({final_size_mb:6.1f} MB)")
    
    total_size = sum(f.stat().st_size for f in checkpoint_dir.rglob('*') if f.is_file()) / (1024*1024)
    print(f"\n📊 Espacio total usado: {total_size:.1f} MB ({total_size/1024:.2f} GB)")
else:
    print(f"⚠️ No se encontró el directorio: {checkpoint_dir}")